# First Test for Volatility Based VaR

In [28]:
import yfinance as yf
import pandas as pd
import numpy as np
from arch import arch_model
from itertools import product

# Import functions
import volatility_var as vv
import backtesting as bt
import plots 
import expected_shortfall as ES
import data_download as dd

### Data

In [29]:
'''# Download data
sp500_data = yf.download("^GSPC", start="2000-01-01", end="2024-01-01")
sp500_data["Log Returns"] = np.log(sp500_data["Close"] / sp500_data["Close"].shift(1))
returns = sp500_data["Log Returns"].dropna()'''

'# Download data\nsp500_data = yf.download("^GSPC", start="2000-01-01", end="2024-01-01")\nsp500_data["Log Returns"] = np.log(sp500_data["Close"] / sp500_data["Close"].shift(1))\nreturns = sp500_data["Log Returns"].dropna()'

In [30]:
# Define tickers and number of shares
tickers = ["MSFT", "NVDA", "AAPL"]
shares = pd.Series({"MSFT": 3, "NVDA": 3, "AAPL": 3})

# Download price data
prices = dd.get_raw_prices(tickers, start="2015-01-01")
prices = dd.convert_to_base(prices, show_currency_detection=False)

# Compute portfolio value each day
portfolio_value = (prices * shares).sum(axis=1)

# Compute simple daily returns
returns = portfolio_value.pct_change().dropna()

# Return as DataFrame (same format as SP500 example)
portfolio_returns_df = returns.to_frame(name="Returns")

returns = portfolio_returns_df["Returns"].dropna()

wealth = portfolio_value.iloc[-1]

returns.head()


Date
2015-01-05   -0.004594
2015-01-06   -0.008707
2015-01-07    0.018517
2015-01-08    0.036281
2015-01-09   -0.001380
Name: Returns, dtype: float64

### Garch based VaR on S&P 500

In [31]:
# Set parameters
confidence_level = 0.99 # <----- Can choose 0.95 etc

In [32]:
# Apply GARCH volatility model
garch_output, next_day_var = vv.var_garch(returns, confidence_level, vol_model="GJR", distribution="skewt")

print(f"Next-day GARCH VaR estimate (abs): {100 * next_day_var:.2f}%")

# Compute ES for the whole period
garch_output = ES.es_volatility(garch_output, confidence_level)

Next-day GARCH VaR estimate (abs): 4.43%


In [33]:
# Plot interactive VaR
fig_var = plots.plot_backtest(garch_output, subset=("2015-11-30", "2020-11-30"), interactive = True)

fig_var_long = plots.plot_backtest(garch_output, interactive = True)

# Plot interactive volatility for a subset
fig_vol = plots.plot_volatility(garch_output["Volatility"], subset=("2015-11-01", "2020-11-30"), interactive=True)

In [34]:
print(next_day_var)

0.04427400656418908


In [35]:
garch_output.head()

,Returns,Volatility,Innovations,VaR,VaR Violation,ES
Date,,,,,,
2015-01-05,-0.004594,0.020688,-0.222064,0.054020,False,0.071332
2015-01-06,-0.008707,0.019832,-0.439043,0.051785,False,0.068382
2015-01-07,0.018517,0.019312,0.958845,0.050426,False,0.066587
2015-01-08,0.036281,0.018471,1.964235,0.048231,False,0.063688
2015-01-09,-0.001380,0.017864,-0.077279,0.046645,False,0.061595


### Arch


In [36]:
# Apply ARCH volatility model
arch_output, next_day_var = vv.var_arch(returns, confidence_level)

print(f"Next-day ARCH VaR estimate (abs): {100 * next_day_var:.2f}%")

# Compute ES for the whole period
arch_output = ES.es_volatility(arch_output, confidence_level)

Next-day ARCH VaR estimate (abs): 4.05%


### Garch Forecasts (with Wealth Scaling)

All works as intended

In [37]:
# Configuration grid
steps_list = [3, 10, 20]
cumulative_list = [False, True]

print("GARCH(1,1) VaR Forecasts (Normal Only)\n" + "-" * 40)

for steps_ahead, cumulative in product(steps_list, cumulative_list):
    try:
        var_value = vv.garch_forecast(
            returns=returns,
            steps_ahead=steps_ahead,
            cumulative=cumulative,
            compute_var=True,
            confidence_level=0.99
        )

        label = f"VaR | {steps_ahead}-day | {'CUM' if cumulative else 'Single'}"
        print(f"{label:<35}: {var_value * 100:.4f}%")  # Convert to percent

    except Exception as e:
        print(f"{label:<35}: ERROR → {e}")


GARCH(1,1) VaR Forecasts (Normal Only)
----------------------------------------
VaR | 3-day | Single               : 6.4399%
VaR | 3-day | CUM                  : 11.2648%
VaR | 10-day | Single              : 6.2321%
VaR | 10-day | CUM                 : 20.2259%
VaR | 20-day | Single              : 5.9722%
VaR | 20-day | CUM                 : 27.9763%


In [38]:
# Test with wealth scaling for VaR
print("\nGARCH(1,1) VaR Forecasts (Scaled by Wealth = $1,000,000)\n" + "-" * 55)

for steps_ahead, cumulative in product(steps_list, cumulative_list):
    try:
        var_value = vv.garch_forecast(
            returns=returns,
            steps_ahead=steps_ahead,
            cumulative=cumulative,
            compute_var=True,
            confidence_level=0.99,
            wealth=1_000_000
        )

        label = f"VaR | {steps_ahead}-day | {'CUM' if cumulative else 'Single'}"
        print(f"{label:<45}: ${var_value:,.2f}")

    except Exception as e:
        print(f"{label:<45}: ERROR → {e}")


# Test that variance forecast with wealth raises an error
print("\nTesting error when computing variance with wealth...\n")

try:
    test = vv.garch_forecast(
        returns=returns,
        steps_ahead=10,
        cumulative=True,
        compute_var=False,  # asking for variance
        wealth=1_000_000     # should raise error
    )
except Exception as e:
    print(f"Expected error: {e}")



GARCH(1,1) VaR Forecasts (Scaled by Wealth = $1,000,000)
-------------------------------------------------------
VaR | 3-day | Single                         : $64,398.86
VaR | 3-day | CUM                            : $112,647.79
VaR | 10-day | Single                        : $62,320.52
VaR | 10-day | CUM                           : $202,258.85
VaR | 20-day | Single                        : $59,721.86
VaR | 20-day | CUM                           : $279,762.63

Testing error when computing variance with wealth...

Expected error: Wealth can only be used when compute_var=True


### MA based VaR on the same data

In [39]:
# Apply MA volatility model
ma_output, next_day_var = vv.var_moving_average(returns, confidence_level)

print(f"Next-day MA VaR estimate (abs): {100 * next_day_var:.2f}%")

# Compute ES for the whole period
ma_output = ES.es_volatility(ma_output, confidence_level)

Next-day MA VaR estimate (abs): 4.92%


In [40]:
# Plot interactive VaR
fig_var = plots.plot_backtest(ma_output, subset = ("2019-11-01", "2020-11-30"), interactive = True, )

### Different volatilities with different models

They look ok, double check later on...

In [41]:
# Apply GARCH volatility model
garch_output_classic, next_day_var_classic = vv.var_garch(returns, confidence_level, vol_model="GARCH")
garch_output_grj, next_day_var_classic = vv.var_garch(returns, confidence_level, vol_model="GJR")
garch_output_egarch, next_day_var_classic = vv.var_garch(returns, confidence_level, vol_model="EGARCH")
garch_output_classic_ged, next_day_var_classic_ged = vv.var_garch(returns, confidence_level, vol_model="GARCH", distribution="ged")
garch_output_classic_skewedt, next_day_var_classic_skewedt = vv.var_garch(returns, confidence_level, vol_model="GARCH", distribution="skewt")

In [42]:
# Plot interactive volatility for a subset
fig_vol_garch = plots.plot_volatility(garch_output_classic["Volatility"], subset=("2019-11-01", "2020-11-30"), interactive=True)
fig_vol_gjr = plots.plot_volatility(garch_output_grj["Volatility"], subset=("2019-11-01", "2020-11-30"), interactive=True)
fig_vol_egarch = plots.plot_volatility(garch_output_egarch["Volatility"], subset=("2019-11-01", "2020-11-30"), interactive=True)
fig_vol_garch_ged = plots.plot_volatility(garch_output_classic_ged["Volatility"], subset=("2019-11-01", "2020-11-30"), interactive=True)
fig_vol_garch_skewedt = plots.plot_volatility(garch_output_classic_skewedt["Volatility"], subset=("2019-11-01", "2020-11-30"), interactive=True)
fig_vol_arch = plots.plot_volatility(arch_output["Volatility"], subset=("2019-11-01", "2020-11-30"), interactive=True)
fig_vol_ma = plots.plot_volatility(ma_output["Volatility"], subset=("2019-11-01", "2020-11-30"), interactive=True)


### Backtesting (Extensive)

In [43]:
# 1. Count violations
total_violations, violation_rate = bt.count_violations(garch_output)

# 2. Kupiec test
kupiec_results = bt.kupiec_test(total_violations, len(garch_output), confidence_level)

# 3. Christoffersen test
christoffersen_results = bt.christoffersen_test(garch_output)

# 4. Joint test
joint_results = bt.joint_lr_test(
    LR_uc=kupiec_results["LR_uc"],
    LR_c=christoffersen_results["LR_c"]
)

# 5. Display results
print("=== VaR Backtesting Summary: GJR-GARCH Model ===")
print(f"Total Violations: {total_violations}")
print(f"Violation Rate: {violation_rate:.2f}%")

print("\n-- Kupiec Test --")
for k, v in kupiec_results.items():
    print(f"{k}: {v}")

print("\n-- Christoffersen Test --")
for k, v in christoffersen_results.items():
    print(f"{k}: {v}")

print("\n-- Joint Test --")
for k, v in joint_results.items():
    print(f"{k}: {v}")


=== VaR Backtesting Summary: GJR-GARCH Model ===
Total Violations: 27
Violation Rate: 1.04%

-- Kupiec Test --
LR_uc: 0.03607110242910494
p_value: 0.8493687336324149
reject_null: False

-- Christoffersen Test --
LR_c: 0.5662239680235643
p_value: 0.4517633245931437
reject_null: False

-- Joint Test --
LR_total: 0.6022950704526693
p_value: 0.7399685932591085
reject_null: False


In [44]:
# 1. Count violations
total_violations, violation_rate = bt.count_violations(ma_output)

# 2. Kupiec test
kupiec_results = bt.kupiec_test(total_violations, len(ma_output), confidence_level)

# 3. Christoffersen test
christoffersen_results = bt.christoffersen_test(ma_output)

# 4. Joint test
joint_results = bt.joint_lr_test(
    LR_uc=kupiec_results["LR_uc"],
    LR_c=christoffersen_results["LR_c"]
)

# 5. Display results
print("=== VaR Backtesting Summary: MA Model ===")
print(f"Total Violations: {total_violations}")
print(f"Violation Rate: {violation_rate:.2f}%")

print("\n-- Kupiec Test --")
for k, v in kupiec_results.items():
    print(f"{k}: {v}")

print("\n-- Christoffersen Test --")
for k, v in christoffersen_results.items():
    print(f"{k}: {v}")

print("\n-- Joint Test --")
for k, v in joint_results.items():
    print(f"{k}: {v}")


=== VaR Backtesting Summary: MA Model ===
Total Violations: 26
Violation Rate: 1.01%

-- Kupiec Test --
LR_uc: 0.0009986809695305965
p_value: 0.97478951698887
reject_null: False

-- Christoffersen Test --
LR_c: 1.2474308366755054
p_value: 0.26404374142971865
reject_null: False

-- Joint Test --
LR_total: 1.248429517645036
p_value: 0.5356819028986742
reject_null: False


### Appendix

#### 1. GARCH(p, q) VaR  
Standard GARCH captures volatility clustering using past squared shocks and variances.

$$
r_t = \mu + \varepsilon_t, \quad \varepsilon_t = \sigma_t z_t
$$

$$
\sigma_t^2 = \omega + \sum_{i=1}^q \alpha_i \varepsilon_{t-i}^2 + \sum_{j=1}^p \beta_j \sigma_{t-j}^2
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 2. EGARCH(p, q) VaR  
EGARCH models log-volatility and captures asymmetry in the volatility response.

$$
\log(\sigma_t^2) = \omega + \sum_{i=1}^q \beta_i \log(\sigma_{t-i}^2) + \sum_{j=1}^p \alpha_j \left( \frac{|\varepsilon_{t-j}|}{\sigma_{t-j}} - \mathbb{E}\left[ \frac{|\varepsilon_{t-j}|}{\sigma_{t-j}} \right] \right) + \gamma_j \frac{\varepsilon_{t-j}}{\sigma_{t-j}}
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 3. GJR-GARCH(p, q) VaR  
GJR-GARCH adds an indicator term to account for the leverage effect of negative shocks.

$$
\sigma_t^2 = \omega + \sum_{i=1}^q \alpha_i \varepsilon_{t-i}^2 + \sum_{j=1}^p \beta_j \sigma_{t-j}^2 + \sum_{i=1}^q \gamma_i \varepsilon_{t-i}^2 \cdot \mathbb{I}_{\{\varepsilon_{t-i}<0\}}
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 4. APARCH(p, q) VaR  
APARCH generalizes GARCH with power terms and asymmetry.

$$
\sigma_t^\delta = \omega + \sum_{i=1}^q \alpha_i \left( |\varepsilon_{t-i}| - \gamma_i \varepsilon_{t-i} \right)^\delta + \sum_{j=1}^p \beta_j \sigma_{t-j}^\delta
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 5. ARCH(p) VaR  
ARCH models volatility using only past squared residuals.

$$
\sigma_t^2 = \omega + \sum_{i=1}^p \alpha_i \varepsilon_{t-i}^2
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 6. EWMA VaR  
EWMA assigns exponentially decaying weights to past squared returns.

$$
\sigma_t^2 = \lambda \sigma_{t-1}^2 + (1 - \lambda) r_{t-1}^2
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 7. MA VaR  
Moving Average volatility is estimated using a rolling window of past squared returns.

$$
\sigma_t = \sqrt{ \frac{1}{n} \sum_{i=1}^n r_{t-i}^2 }
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 8. Expected Shortfall (ES)  
Expected Shortfall estimates the conditional expectation of loss beyond the VaR.

$$
\text{ES}_t = -\hat{\sigma}_t \cdot \mathbb{E}[z_t \mid z_t < z_\alpha]
$$

---

#### 9. GARCH(1,1) Variance Forecast – Analytical  

Forecast for single-step variance $T$ steps ahead:

$$
\mathbb{E}[\sigma_{t+T}^2] = \text{VL} + (\alpha + \beta)^T \cdot (\sigma_t^2 - \text{VL})
$$

Forecast for cumulative variance over $T$ days:

$$
\mathbb{E}[\sigma_{t,T}^2] =
\text{VL} \cdot \left( T - 1 - \frac{(\alpha + \beta)(1 - (\alpha + \beta)^{T - 1})}{1 - (\alpha + \beta)} \right)
+ \sigma_t^2 \cdot \frac{1 - (\alpha + \beta)^T}{1 - (\alpha + \beta)}
$$

Where:

- $\text{VL} = \dfrac{\omega}{1 - \alpha - \beta}$ is the long-run variance  
- $\sigma_t^2$ is the last fitted conditional variance  


#### 10. GARCH(1,1) VaR Forecast – Analytical  

Once variance is forecasted, the VaR over the horizon $T$ is:

$$
\text{VaR}_{t,T} = - z_\alpha \cdot \sqrt{\mathbb{E}[\sigma_{t,T}^2]}
$$

Where $z_\alpha$ is the quantile of the fitted standardized innovation distribution:

- Normal: $z_\alpha = \Phi^{-1}(1 - \alpha)$  
- Student-t / GED: fitted on standardized residuals
